In [27]:
import traceback
import urllib3
import xmltodict
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import geocoder
import requests
import math

In [28]:

def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return (abs(c * r))

def getxml():
    url = "https://www.fuelwatch.wa.gov.au/fuelwatch/fuelWatchRSS?"
    http = urllib3.PoolManager()

    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    response = http.request('GET', url, headers={'User-Agent':user_agent,})

    try:
        data = xmltodict.parse(response.data)
    except:
        print("Failed to parse xml from response (%s)" % traceback.format_exc())
    return data

In [29]:
data = getxml()

In [30]:
df = pd.DataFrame(data['rss']['channel']['item'])


df['latitude'] = df['latitude'].astype('double')
df['longitude'] = df['longitude'].astype('double')

In [31]:
g = geocoder.ip("me")
CURRENT_LAT = g.latlng[0]
CURRENT_LON = g.latlng[1]
df['distance_from_me'] = df.apply(lambda row: distance(CURRENT_LAT, row['latitude'], CURRENT_LON, row['longitude']), axis=1)

In [32]:
MAX_DISTANCE = 10

In [33]:
# ,
stations = df[['title', 'description', 'price', 'distance_from_me']]
stations[stations['distance_from_me'] < MAX_DISTANCE].sort_values(by='price')

,title,description,price,distance_from_me
7,189.9: BP Connect Myaree,"Address: 246 Leach Hwy, MYAREE, Phone: (08) 93...",189.9,7.221943
75,193.9: BP Connect Wembley,"Address: 240 Cambridge St, WEMBLEY, Phone: (08...",193.9,4.743280
67,193.9: BP Highgate,"Address: 342 Beaufort St, PERTH, Phone: 0450 7...",193.9,7.151479
65,193.9: BP Connect Main Street,"Address: 218 Main St, OSBORNE PARK, Phone: (08...",193.9,9.718826
62,193.9: Ampol Rous Head,"Address: Cnr North Mole Dr & Rous Head Rd, NOR...",193.9,9.230765
...,...,...,...,...
365,199.9: Ampol Willetton,"Address: 169 High Rd, WILLETTON, Phone: (08) 9...",199.9,9.643149
370,201.5: Vibe Subiaco,"Address: 123 Thomas St, SUBIACO, Phone: (08) 9...",201.5,3.977018
371,201.7: Puma Burswood,"Address: 265 Great Eastern Hwy, BURSWOOD, Phon...",201.7,9.237315
376,201.9: 7-Eleven Booragoon,"Address: 492 Marmion St, BOORAGOON, Phone: (08...",201.9,6.586255


In [34]:
df.sort_values('distance_from_me')

,title,description,brand,date,price,trading-name,location,address,phone,latitude,longitude,site-features,distance_from_me
350,199.9: Ampol Foodary Nedlands,"Address: 7 Broadway, NEDLANDS, Phone: (08) 638...",Ampol,2022-07-09,199.9,Ampol Foodary Nedlands,NEDLANDS,7 Broadway,(08) 6389 1898,-31.977620,115.813656,", Open 24 hours",0.758570
60,193.9: BP Rosegarden,"Address: 129 Stirling Hwy, NEDLANDS, Phone: (0...",BP,2022-07-09,193.9,BP Rosegarden,NEDLANDS,129 Stirling Hwy,(08) 9386 1014,-31.979719,115.798235,", Open 24 hours",0.885756
215,197.9: Coles Express Claremont,"Address: 269 Stirling Hwy, CLAREMONT, Phone: (...",Coles Express,2022-07-09,197.9,Coles Express Claremont,CLAREMONT,269 Stirling Hwy,(08) 6247 4366,-31.982195,115.783890,", Open 24 hours",2.208401
405,214.9: United Swanbourne,"Address: 2 Servetus St, SWANBOURNE, Phone: (08...",United,2022-07-09,214.9,United Swanbourne,SWANBOURNE,2 Servetus St,(08) 9384 2944,-31.983608,115.767278,"Conditional Driveway Service, Air Bottled Gas ...",3.780082
360,199.9: Ampol Foodary Subiaco,"Address: 194 Rokeby Rd, SUBIACO, Phone: (08) 9...",Ampol,2022-07-09,199.9,Ampol Foodary Subiaco,SUBIACO,194 Rokeby Rd,(08) 9388 1701,-31.950616,115.824234,", Open Mon-Fri: 05:00-23:59, Sat: 05:00-23:59,...",3.817773
...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,209.9: Karnup Roadhouse,"Address: 2263 Mandurah Rd, KARNUP, Phone: 0476...",BP,2022-07-09,209.9,Karnup Roadhouse,KARNUP,2263 Mandurah Rd,0476 913 075,-32.427429,115.774670,", Open 24 hours",49.646698
72,193.9: BP The Lakes Roadhouse,"Address: 13724 Great Eastern Hwy, THE LAKES, P...",BP,2022-07-09,193.9,BP The Lakes Roadhouse,THE LAKES,13724 Great Eastern Hwy,(08) 9572 6025,-31.875285,116.320602,", Open 24 hours",49.868215
131,195.9: Ampol Foodary Yanchep,"Address: 1 Morwell St, YANCHEP, Phone: (08) 95...",Ampol,2022-07-09,195.9,Ampol Foodary Yanchep,YANCHEP,1 Morwell St,(08) 9554 1265,-31.552063,115.639632,", Open 24 hours",50.344957
175,197.5: Puma Yanchep,"Address: 1 Ikara Lane, YANCHEP, Phone: (08) 95...",Puma,2022-07-09,197.5,Puma Yanchep,YANCHEP,1 Ikara Lane,(08) 9506 4337,-31.547779,115.641352,", Open Mon-Fri: 05:00-22:00, Sat: 05:00-22:00,...",50.746920


In [35]:
df[df['distance_from_me'] < 5].sort_values('distance_from_me')

,title,description,brand,date,price,trading-name,location,address,phone,latitude,longitude,site-features,distance_from_me
350,199.9: Ampol Foodary Nedlands,"Address: 7 Broadway, NEDLANDS, Phone: (08) 638...",Ampol,2022-07-09,199.9,Ampol Foodary Nedlands,NEDLANDS,7 Broadway,(08) 6389 1898,-31.977620,115.813656,", Open 24 hours",0.758570
60,193.9: BP Rosegarden,"Address: 129 Stirling Hwy, NEDLANDS, Phone: (0...",BP,2022-07-09,193.9,BP Rosegarden,NEDLANDS,129 Stirling Hwy,(08) 9386 1014,-31.979719,115.798235,", Open 24 hours",0.885756
215,197.9: Coles Express Claremont,"Address: 269 Stirling Hwy, CLAREMONT, Phone: (...",Coles Express,2022-07-09,197.9,Coles Express Claremont,CLAREMONT,269 Stirling Hwy,(08) 6247 4366,-31.982195,115.783890,", Open 24 hours",2.208401
405,214.9: United Swanbourne,"Address: 2 Servetus St, SWANBOURNE, Phone: (08...",United,2022-07-09,214.9,United Swanbourne,SWANBOURNE,2 Servetus St,(08) 9384 2944,-31.983608,115.767278,"Conditional Driveway Service, Air Bottled Gas ...",3.780082
360,199.9: Ampol Foodary Subiaco,"Address: 194 Rokeby Rd, SUBIACO, Phone: (08) 9...",Ampol,2022-07-09,199.9,Ampol Foodary Subiaco,SUBIACO,194 Rokeby Rd,(08) 9388 1701,-31.950616,115.824234,", Open Mon-Fri: 05:00-23:59, Sat: 05:00-23:59,...",3.817773
334,199.9: Coles Express Jolimont,"Address: 6 Jersey St, JOLIMONT, Phone: (08) 62...",Coles Express,2022-07-09,199.9,Coles Express Jolimont,JOLIMONT,6 Jersey St,(08) 6247 4374,-31.947273,115.811941,", Open Mon-Fri: 06:00-22:00, Sat: 06:00-21:00,...",3.864110
370,201.5: Vibe Subiaco,"Address: 123 Thomas St, SUBIACO, Phone: (08) 9...",Vibe,2022-07-09,201.5,Vibe Subiaco,SUBIACO,123 Thomas St,(08) 9382 2281,-31.952307,115.831150,", Open Mon-Fri: 05:00-21:00, Sat: 06:00-20:00,...",3.977018
264,197.9: Coles Express Wembley,"Address: 337 Cambridge St, WEMBLEY, Phone: (08...",Coles Express,2022-07-09,197.9,Coles Express Wembley,WEMBLEY,337 Cambridge St,(08) 6247 4384,-31.941023,115.811858,", Open Mon-Fri: 06:00-22:00, Sat: 06:00-22:00,...",4.554539
75,193.9: BP Connect Wembley,"Address: 240 Cambridge St, WEMBLEY, Phone: (08...",BP,2022-07-09,193.9,BP Connect Wembley,WEMBLEY,240 Cambridge St,(08) 9381 1000,-31.940547,115.820095,"Conditional Driveway Service, Open 24 hours",4.743280


In [36]:
df.brand.unique()

array(['Atlas', 'BP', 'Shell', 'Caltex', 'United', 'Vibe', 'Independent',
       'Puma', 'Costco', 'Ampol', 'Liberty', 'Coles Express',
       'Better Choice', 'FastFuel 24/7', 'Caltex Woolworths', '7-Eleven',
       'EG Ampol'], dtype=object)

In [51]:
def bestStation(startPos, endPos, stations, maxDiversion=5, km_per_l=16.5, desired_amt=50, currAmt = 1, hasRACDiscount=False,hasWoolieDiscount=False):
    distances = []
    RACStations = set(['Puma','Caltex','Better Choice'])
    WooliesStations = set(['Ampol','EG Ampol','Caltex','Caltex Woolworths'])
    for index, row in stations.iterrows():        
        station = []
        station.append(row['address'])
       
        if hasRACDiscount and hasWoolieDiscount:
            if row['brand'] in RACStations.union(WooliesStations):
                row['price'] = str(float(row['price']) - 4)

        elif hasRACDiscount:
            if row['brand'] in RACStations:
                row['price'] = str(float(row['price']) - 4)

        elif hasWoolieDiscount:
            if row['brand'] in WooliesStations:
                row['price'] = str(float(row['price']) - 4)

        
        d1 = distance(startPos[0], row['latitude'], startPos[1], row['longitude'])
        d2 = distance(endPos[0], row['latitude'], endPos[1], row['longitude'])
        d3 = distance(startPos[0], endPos[0], startPos[1], endPos[1])
        diversion = d1+d2 - d3
        
        
        # Filter out locations we cant reach with our current fuel
        if(d1 > currAmt*km_per_l):
            continue
        
        # Filter out locations above our max diversion distance
        if(maxDiversion < diversion):
            continue
        
        tank_at_servo = currAmt - d1 * (1/km_per_l)
        spent_at_servo = (desired_amt - tank_at_servo) * float(row['price'])
        final_leg_price = float(row['price']) # assumption
        final_leg_cost  = d2 * (1/km_per_l) * final_leg_price
        
        total_cost = spent_at_servo + final_leg_cost
        
        station.append(diversion)
        
        station.append(total_cost)
        
        distances.append(station)
        
    sorted(distances, key=lambda r: r[2])    
    return distances[0]

In [61]:
def bestStationOnPath(pathPoints, stations, maxDiversion=5, km_per_l=16.5, desired_amt=50, currAmt = 1):
    distances = []
    for i in range(len(pathPoints)-1):
        distance = bestStation(pathPoints[i], pathPoints[i+1], stations, maxDiversion, km_per_l, desired_amt, currAmt)
        distances.append(distance)
    sorted(distances, key=lambda r: r[2])
    return distances[0]

In [64]:
path=[(-31.979532,115.816404),(-31.957913,115.848805),(-32.009361,115.856199)]
bestStationOnPath(path,df)

['129 Stirling Hwy', 3.176482319557273, 9584.124351512328]

In [53]:
destination = [-32.315863, 115.752395]
distances = bestStation(g.latlng, destination, df)
sorted(distances, key=lambda r: r[2])

TypeError: 'float' object is not subscriptable

In [107]:
wayPoint1 = f"{CURRENT_LAT},{CURRENT_LON}"
wayPoint2 = f"{-31.977620},{115.813656}"
optimize = 'distance'
BingMapsKey = 'Ah2oroOW1tCkD7JahjO6Ohfy6rDnYUdaweMCUiSk3k5vdyRi6CkR1nd9xThKml5B'
url = f"http://dev.virtualearth.net/REST/v1/Routes?waypoint.1={wayPoint1}&waypoint.2={wayPoint2}&optimize={optimize}&key={BingMapsKey}"
response = requests.get(url)


In [108]:
response.json()

{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2022 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.',
 'resourceSets': [{'estimatedTotal': 1,
   'resources': [{'__type': 'Route:http://schemas.microsoft.com/search/local/ws/rest/v1',
     'bbox': [-31.981798, 115.8077, -31.977552, 115.814108],
     'id': 'v70,h7458028,i0,a0,cen-US,dAAAAAAAAAAA1,y0,s1,m1,o2,t4,wejarPlf7P8Dkg57NqvNcQA2~AM38GuGJp-rKAADgAcLLmD4C0~VGhvbWFzIFN00~~~~~~~~v12,wYOrnTUX6P8BtjJ3wEvRcQA2~AM38GuEZmOrKAADgAWjpJj8B0~QnJvYWR3YXk1~~~~~~~~v12,k1',
     'distanceUnit': 'Kilometer',
     'durationUnit': 'Second',
     'routeLegs': [{'actualEnd': {'type': 'Point',
        'coordinates': [-31.977552, 115.81391]},
       'actualStart': {'type': 